### Bar chart

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
# import matplotlib

# # 使用 FontProperties 設定字體
# matplotlib.rc('font', family='Microsoft JhengHei')
# # 假設你的 DataFrame 叫做 df，並且地點的欄位名稱為 '地點'
# locations = sorted(df['樣點編號'].unique())

# fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# for i, location in enumerate(locations):
#     ax = axs[i//3, i%3]
#     data = df[df['樣點編號'] == location].groupby('鳥種')['數量'].sum().sort_values(ascending=False)
#     data.plot(kind='bar', ax=ax, color='C4')
#     ax.set_title(location)
#     ax.set_ylabel('數量')

# plt.tight_layout()
# plt.style.context('seaborn-whitegrid')
# plt.show()

### Chord map

In [ ]:
# Install
# 可以用 但吃率 html
# "C:\Users\harew\AppData\Local\Temp\d3blocks\chord.html"
bird_place_num = df.groupby(['鳥種', '樣點編號'])['數量'].sum().reset_index(name='數量')
bird_place_num
# Load d3blocks
from d3blocks import D3Blocks

# Initialize
d3 = D3Blocks()

# Load example data
# df = d3.import_example('energy')

# Plot
bird_place_num.rename(columns={'鳥種': 'source', '樣點編號': 'target', '數量': 'value'}, inplace=True)
d3.chord(bird_place_num, title='Chord Diagram of Bird Species and Places')

### Line plot

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 將 '年', '月', '日', '開始時間\n（時）', '開始時間\n（分）' 這些欄位組合成一個 datetime 對象
df['開始時間\n（時）'] = df['開始時間\n（時）'].astype(int)
df['開始時間\n（分）'] = df['開始時間\n（分）'].astype(int)
df['年'] = df['年'].astype(int)
df['月'] = df['月'].astype(int)
df['日'] = df['日'].astype(int)
df['datetime'] = pd.to_datetime(df['年'].astype(str) + '-' + df['月'].astype(str) + '-' + df['日'].astype(str) + ' ' + df['開始時間\n（時）'].astype(str) + ':' + df['開始時間\n（分）'].astype(str))
df['datetime_cp'] = pd.to_datetime(df['datetime'])
# 將 datetime 列設置為索引
df.set_index('datetime', inplace=True)
df_mornong = df.between_time('06:00', '12:00')
df_afternoon = df.between_time('12:00', '18:00')

# 對每個地點和時間點計算鳥種數量
bird_counts = df_mornong.groupby(['datetime_cp', '樣點編號'])['鳥種'].nunique()
bird_counts = bird_counts.reset_index(name='鳥種數量')

# 定義日期範圍
bins = pd.date_range(start='2024-03-13 08:00:00', end='2024-05-21 23:38:00', freq='7D')
# 創建新的欄位 'week'
bird_counts['week'] = pd.cut(bird_counts['datetime_cp'], bins=bins, labels=range(1, len(bins)), right=True)

# 繪製折線圖
fig, ax = plt.subplots()

for location in bird_counts['樣點編號'].unique():
    location_data = bird_counts[bird_counts['樣點編號'] == location]
    weekly_counts = location_data.groupby('week')['鳥種數量'].mean()
    ax.plot(weekly_counts.index, weekly_counts.values, label=location)

# 設置 x 軸的標籤

ax.set_xlabel('Week')

# 旋轉 x 軸的標籤
plt.xticks(rotation=90)

plt.legend()
plt.show()